# ⭐ radix sort를 구현해보자.

## 고정된 3자리 정수 라면

In [ ]:
arr = [329, 457, 657, 839, 436, 720, 355]

### **1의 자리 정렬**
#### > 주의 사항
 - bucket을 만들때, 자릿수별로 만들어야 하니 미리 0~9 까지 만들어 놓고, 해당 자리수에 넣어야 하기 때문에 이중리스트로
 - 이중리스트를 풀어서 원소만 넣을것이기 때문에 그대로 추가하는 append 대신 extend로 풀어서 넣기

In [ ]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = x%10
  bucket[digit].append(x)

arr = []

for b in bucket:
  arr.extend(b)

print(arr)

[720, 355, 436, 457, 657, 329, 839]


### 10의 자리 정렬

In [ ]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = (x//10)%10
  bucket[digit].append(x)

arr = []
for b in bucket:
  arr.extend(b)

print(arr)

[720, 329, 436, 839, 355, 457, 657]


### 100의 자리 정렬

In [ ]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = (x//100)%10
  bucket[digit].append(x)

arr = []

for b in bucket:
  arr.extend(b)

print(arr)

[329, 355, 436, 457, 657, 720, 839]


## 가변길이 정수라면

In [ ]:
arr = [38295, 291, 692602, 293, 291, 12, 0, 12985]

In [ ]:
max_len = len(str(max(arr)))

In [ ]:
for i in range(max_len):
  bucket = [[] for _ in range(10)]
  div = 10**i
  for x in arr:
    digit = (x//div) %10
    bucket[digit].append(x)

  arr = []

  for b in bucket:
    arr.extend(b)

In [ ]:
print(arr)

[0, 12, 291, 291, 293, 12985, 38295, 692602]


## ❤ Radix Sort 구현하며 겪은 핵심 실수 정리

---

## 1. bucket 구조 오해

❌ 잘못된 생각
`bucket = []`
임시 저장소 하나만 있으면 된다고 생각함.

⭕ 올바른 구조
`bucket = [[] for _ in range(10)]`
→ 0~9 자릿수별 분류함 10개가 필요함.

---


## 2. append 와 extend 혼동
❌ 실수
`arr.append(b)`
→ 리스트 안에 리스트가 들어가 구조 붕괴.

⭕ 정답
`arr.extend(b)`
→ 버킷 안의 값들을 풀어서 삽입.

---

## 3. bucket 초기화 위치 오류
❌ 실수
`bucket = [[] for _ in range(10)]   # for 밖`
→ 이전 단계 데이터가 계속 누적되어 값 폭증.

⭕ 정답
```python
for exp in range(max_len):
    bucket = [[] for _ in range(10)]   # 반드시 for 안
```

---

## 4. 1의 자리 정렬 생략
❌ 실수
`for i in range(1, max_len):`
→ 10의 자리부터 시작 → 안정 정렬 깨짐 → 중복 순서 붕괴.

⭕ 정답
`for exp in range(max_len):`

---


## 5. 내림차순 구현 착각
❌ 실수
`for b in reversed(bucket):`
→ LSD 방식에서 안정성 파괴.

⭕ 정답
오름차순으로 완성 후
`arr.reverse()`

---


## 6. arr = bucket 대입 실수
❌ 실수
`arr = bucket`
→ 2차원 구조로 변해 다음 단계에서 에러.

⭕ 정답
```python
arr = []
for b in bucket:
    arr.extend(b)
```

# ⭐ Floyd-Warshall 을 구현해보자.

## ▶ **Floyd-Warshall 알고리즘** ?

모든 정점 쌍 사이의 최단 거리를 구하는 알고리즘

> 언제써?
> - "A에서 B까지 최단 거리”
> - "B에서 C까지 최단 거리”
> - 이런 걸 모든 쌍에 대해 전부 알고 싶을 때 사용한다.


## ▶ 이차원 리스트의 의미

`dist[i][j]` : `i`번 정점에서 `j`번 정점까지 가는 **현재까지 알고 있는 최단 거리**

> 초기 상태는
> - 자기 자신 -> `0`
> - 간언이 있으면 -> 그 가중치
> - 없으면 -> 매우 큰 값($\infty$)

## ▶ 핵심 아이디어

> `i -> j`로 가는것보다 `i -> k -> j`로 우회하는게 더 짧으면 바꾼다.

이걸 **모든 `i` 모든 `j` 모든 `k`** 조합에 대해 반복

In [ ]:
INF = int(1e9)
n = 3
m = 2

In [ ]:
# graph 없을때 n, m 입력받기용
# graph = [[INF]*(n+1) for _ in range(n+1)]

# 자기 자신으로 가는 경우를 0 으로 초기화
# for a in range(len(graph)):
#   for b in range(len(graph)):
#     if a == b:
#       graph[a][b] = 0

# 초기 데이터
graph = [
    [0,   3,  float('inf'), 7],
    [8,   0,  2,            float('inf')],
    [5,   float('inf'), 0,  1],
    [2,   float('inf'), float('inf'), 0]
]

In [ ]:
%%time
for i in range(len(graph)):
  for j in range(len(graph)):
    for k in range(len(graph)):
      graph[i][j] = min(graph[i][j], graph[i][k] + graph[k][j])

In [ ]:
graph

## ❤ 피드백

> k번째 반복이 끝났을 때 `graph[i][j]` 는 중간 노드로 `0 ~ k`까지만 사용하는 단거리 여야 한다.

이 조건을 만족하려면
k가 가장 바깥 루프에 있어야 한다.

In [ ]:
n = len(graph)

In [ ]:
%%time
for k in range(n):          # 거쳐가는 노드
    for i in range(n):      # 출발 노드
        for j in range(n):  # 도착 노드
            if graph[i][j] > graph[i][k] + graph[k][j]:
                graph[i][j] = graph[i][k] + graph[k][j]


In [ ]:
graph

# ⭐ Gradient Descent(경사하강법)을 하드코딩 해보자!

---

## ▶ Gradient Descnet?
> 함수의 기울기(미분값)을 이용하여 함수값을 가장 빠르게 감소시키는 방향으로 조금씩 이동하면서 최소값을 찾는 최적화 알고리즘

---

## ▶ 왜 **기울기 반대 방향**?
> 기울기는 현재 위치에서 **함수값이 가장 빠르게 증가하는 방향**이기 때문에

따라서 `- 기울기 방향`은 함수값이 *가장 빠르게 감소하는 방향* 이 된다.

---

## ▶ 수식으로 정리해볼까?

- 단변수
$$ x_{t+1}​=x_t​−α⋅f'(x_t​) $$

- 다변수
$$ W_{t+1} = W_t - α⋅\nabla f(W_t) $$

|기호|의미|
|---|---|
|$~~~~~~~~α~~~~~~~~$| 학습률 (step size)|
|$$ f'(x)  $$ | 기울기|
|$$ \nabla f$$ | 모든 변수에 대한 기울기 벡터|

---

## ▶ 학습률 ( $α~$)
학습률은 **한 번 이동할 때 얼마나 크게 이동할지 결정하는 값**이다.
|크기|현상|
|---|---|
|너무 큼|최소값을 넘어 튕기며 발산|
|적당함|안정적으로 최소값 수렴|
|너무 작음|수렴은 하지만 매우 느림|

---

## ▶ 정답을 찾지 못하는 이유

### (1) 지역 최솟값 문제
경사하강법은 출발 위치 근처의 지역 최솟값(Local Minimum)에 빠질 수 있다.

### (2) 평평한 구간 문제 (Plateau)
기울기가 거의 $0$인 평평한 영역에서는 이동 속도가 극도로 느려진다.

### (3) 안장점 (Saddle Point)
기울기는 $0$이지만 최소도 최대도 아닌 지점에서 멈춰버릴 수 있다.

---

## ▶ Gradient Descent 종류
|종류|설명|
|---|---|
|batch GD| 전체 데이터로 한 번에 기울기 계산|
|SGD| 데이터 하나마다 기울기 계산|
|Mini-batch GD| 일부 묶음 단위로 계산|



## ❤ 구현 시 주의사항

---

### ① 학습률 범위

만약 $ f(x) = 3x^2 + 0.1x $ , $ f'(x) = 6x + 0.1 $

```python
def function_1(x):
  return 3 * x**2 + 0.1 * x
```

초기값 $ x=2$ 이면 $ f'(2) = 12.1$

경사하강 $1$ 스텝 : $ x_{new} = 2-10⋅12.1 = -119 $

첫 스텝에서 기울기의 10배를 넘겨버림 -> 발산 -> 미쳐 날뛰기 시작.

> 이 예시의 경우 **이차함수(볼록 함수)** 라서
> 안정적인 α는 보통 `0.01 ~ 0.001` 수준으로 잡는다.

---

### ② 수치미분 h의 값

$ h=1$ 은 너무 크다.
```python
def diff(f,x):
  h = 1
  return (f(x+h)-f(x-h)) / h
```
미분 수준이 아니라 흉내 낸거라 기울기가 계속 **과대추정**된다.

---

### ③ 발산

$ x_{t+1} = x_t - 10⋅(6x_t + 0.1)  $  
➡ $~ x_{t+1} = -59x_t -1 $  
➡  $~ |-59|>1$

> 수열이 **지수적으로 폭발하도록 설계**된 수식

---

### ④ 진짜 최솟값은?

$ f'(x) = 6x+0.1 = 0 $ ➡ $ x= -0.016666~ ... $

---

### ⑤ 정리

|항목|현재 상태|문제|
|---|---|---|
| 학습률|$10$|너무 큼 -> 발산|
|수리미분 $h$| $1$ | 기울기 과대
|함수 형태| 2차 함수| 작은 학습률 필수|

## 컨셉 1

```
초기값 x 를 하나 정한다
학습률 alpha 를 정한다
반복 횟수 max_iter 를 정한다

for 반복횟수 만큼:
    현재 기울기 = f'(x) 계산
    x = x - alpha * 현재 기울기
    x 값 기록 (필요하면)

```

In [ ]:
def diff(f,x):
  h = 1
  return (f(x+h)-f(x-h)) / h

In [ ]:
def function_1(x):
  return 3 * x**2 + 0.1 * x

In [ ]:
diff(function_1, 2)

24.2

In [ ]:
x = 2
alpha = 10
max_iter = 5
answer = []

In [ ]:
for _ in range(max_iter):
  diff_value = diff(function_1, x)
  x = x - alpha * diff_value
  answer.append(x)

In [ ]:
answer

[-240.0,
 28557.999999999884,
 -3398404.000002861,
 404410074.00780964,
 -48124798245.99219]

실패 한 것

---

In [ ]:
def gd(f, grad_f, x, alpha, max_iter):
  answer = []

  for _ in range(max_iter):
    g = grad_f(x)
    x = x - alpha * g
    answer.append(x)

  return x, answer

In [ ]:
def f1(x):
  return 3*x**2 + 0.1*x

In [ ]:
def grad_f1(x):
  return 6*x+0.1

In [ ]:
x0=2
x_min, trace = gd(f1, grad_f1, x0, alpha=0.01, max_iter=100)

In [ ]:
x_min

-0.012522669212777409

## 컨셉 2

```
초기값 x 설정
학습률 alpha 설정
허용 오차 epsilon 설정

while True:
    현재 기울기 = f'(x)
    
    만약 |현재 기울기| < epsilon 이면:
        break

    x = x - alpha * 현재 기울기
```

In [ ]:
def gradient_descent_until(f, grad_f, x, alpha=0.01, epsilon=1e-6):
    history = []

    while True:
        g = grad_f(x)

        if abs(g) < epsilon:
            break

        x = x - alpha * g
        history.append(x)

    return x, history

In [ ]:
x0 = 2
x_min, trace = gradient_descent_until(f1, grad_f1, x0)

In [ ]:
x_min

-0.016666504340897184

## 컨셉 3 다변수

```
초기 벡터 W 설정
학습률 alpha 설정

for 반복:
    gradient = 모든 변수에 대한 편미분 벡터
    W = W - alpha * gradient
```

In [ ]:
pip install numpy

In [ ]:
import numpy as np

In [ ]:
def gradient_descent_multi(grad_f, W, alpha=0.01, max_iter=100):
    history = []

    for _ in range(max_iter):
        grad = grad_f(W)
        W = W - alpha * grad
        history.append(W.copy())

    return W, history

In [ ]:
# f(x, y) = x^2 + y^2
def grad_f2(W):
    x, y = W
    return np.array([2*x, 2*y])

In [ ]:
W0 = np.array([5.0, -3.0])
W_min, trace = gradient_descent_multi(grad_f2, W0)

In [ ]:
W_min

array([ 0.66309778, -0.39785867])

## 결과

- 단변수는 $x≈-0.01666~...$ 근처로 수렴
- 다변수는 `[0, 0]`으로 수렴